<a href="https://colab.research.google.com/github/AtlasEffect/bachata-video-proffessor/blob/main/ACE_Step_1_5_Colab_UI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🎵 ACE-Step v1.5 - Google Colab Launcher
### 🔴 Brought to you by [AI With Chucky](https://youtube.com/@AIWithChucky)

**Notes:**
- This requires a GPU runtime (T4 is usually sufficient).
- The first run will take a few minutes to download the models (approx 10GB).
- **Wait for the link ending in `gradio.live` to appear at the bottom.**

In [1]:
# @title 1. Clone Repository & Patch Dependencies
import os
import sys

# 1. Clone the Hugging Face Space
if not os.path.exists("Ace-Step-v1.5"):
    print("Cloning ACE-Step v1.5 repository...")
    !git clone https://huggingface.co/spaces/ACE-Step/Ace-Step-v1.5
else:
    print("Repository already exists.")

# 2. Force Change Directory
# This ensures subsequent commands run inside the folder
os.chdir("/content/Ace-Step-v1.5")
print(f"Working directory set to: {os.getcwd()}")

# 3. Patch requirements.txt
# The original file asks for torch>=2.9.1 (future version) for Linux, which fails install.
# We relax this to allow the current stable torch version.
print("Patching requirements.txt...")
!sed -i 's/torch>=2.9.1/torch/g' requirements.txt

# 4. Patch app.py for Public Access
# By default, share=False. We change it to share=True to get a public Gradio link.
print("Patching app.py to enable public sharing...")
!sed -i 's/share=False/share=True/g' app.py

print("✅ Setup phase 1 complete.")

Repository already exists.
Working directory set to: /content/Ace-Step-v1.5
Patching requirements.txt...
Patching app.py to enable public sharing...
✅ Setup phase 1 complete.


In [2]:
# @title 2. Install Dependencies
import os

# Ensure we are in the correct folder
if os.getcwd() != "/content/Ace-Step-v1.5":
    os.chdir("/content/Ace-Step-v1.5")

print("Installing Python dependencies... (This may take 3-5 minutes)")

# Install dependencies with CUDA support
# We skip strict version checks for some libs to avoid Colab conflicts
!pip install -r requirements.txt --extra-index-url https://download.pytorch.org/whl/cu121

# Install system dependency for audio processing
!apt-get install -y ffmpeg

print("✅ Dependencies installed.")

Installing Python dependencies... (This may take 3-5 minutes)
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu121
Ignoring triton-windows: markers 'sys_platform == "win32"' don't match your environment
Ignoring flash-attn: markers 'sys_platform == "win32" and python_version == "3.11" and platform_machine == "AMD64"' don't match your environment
Ignoring flash-attn: markers 'sys_platform == "linux" and python_version == "3.11"' don't match your environment
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.
✅ Dependencies installed.


In [3]:
%cd /content/Ace-Step-v1.5
!pip install -e ./acestep/third_parts/nano-vllm

/content
Obtaining file:///content/Ace-Step-v1.5/acestep/third_parts/nano-vllm
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for nano-vllm (pyproject.toml) ... done
  Created wheel for nano-vllm: filename=nano_vllm-0.2.0-0.editable-py3-none-any.whl size=5037 sha256=bd5673f69c087c922b32048198fbd3965b32e18b5b52043fa4f5e3fff17e3722
  Stored in directory: /tmp/pip-ephem-wheel-cache-atg1s5w5/wheels/0c/95/fd/55ed745fe219a1900e427f7db76e5eef5c2e3c3e0cfdaa8a7e
Successfully built nano-vllm
  Attempting uninstall: nano-vllm
    Found existing installation: nano-vllm 0.2.0
    Uninstalling nano-vllm-0.2.0:
      Successfully uninstalled nano-vllm-0.2.0


In [9]:
import json
import os

model_dir = "/content/Ace-Step-v1.5/data/checkpoints/acestep-5Hz-lm-1.7B"
config_path = os.path.join(model_dir, "config.json")

if os.path.exists(config_path):
    with open(config_path, 'r') as f:
        config = json.load(f)

    # Remove or set to false any flash attn flags
    if "attn_implementation" in config:
        del config["attn_implementation"]  # or set to "eager"
    if "_attn_implementation" in config:  # sometimes hidden
        del config["_attn_implementation"]
    config["use_flash_attn"] = False       # if present
    config["attn_implementation"] = "eager"  # or "sdpa"

    with open(config_path, 'w') as f:
        json.dump(config, f, indent=2)

    print("✅ Patched config.json to disable flash attention")
else:
    print("config.json not found — skip patch")

✅ Patched config.json to disable flash attention


In [ ]:
# @title 3. Download Models & Launch Interface
# This cell first downloads the heavy model files (safetensors) and then starts the app.
import os
os.environ["USE_FLASH_ATTENTION"] = "0"

os.environ["VLLM_USE_FLASH_ATTN"] = "0"                  # Upstream vLLM var
os.environ["VLLM_ATTENTION_BACKEND"] = "TORCH_SDPA"      # ← Key: force torch's built-in scaled_dot_product_attention (no flash)
# Optional extra (some repos respect this):
os.environ["FLASH_ATTENTION"] = "0"
if os.getcwd() != "/content/Ace-Step-v1.5":
    os.chdir("/content/Ace-Step-v1.5")

# --- LITE MODE CONFIG ---
os.environ["SERVICE_MODE_DIT_MODEL_2"] = "" # Disable 2nd model download/load
os.environ["SERVICE_MODE_BACKEND"] = "pt"

# 1. CREATE DOWNLOAD SCRIPT
# Note: Double backslashes (\\n) are used to prevent Python from breaking strings across lines in the file
download_code = """
import os
os.environ["USE_FLASH_ATTENTION"] = "0"
os.environ["VLLM_USE_FLASH_ATTN"] = "0"
os.environ["FLASH_ATTENTION"] = "0"
os.environ["ATTN_BACKEND"] = "torch"
os.environ["TORCH_USE_CUDA_SDPA"] = "1"
import sys
import torch

print("Initializing download sequence...")
current_dir = os.getcwd()
sys.path.insert(0, os.path.join(current_dir, "acestep", "third_parts", "nano-vllm"))

from acestep.handler import AceStepHandler

# Initialize handler to trigger download logic
# We force CPU offload to avoid filling GPU RAM just for downloading
handler = AceStepHandler(persistent_storage_path=os.path.join(current_dir, "data"))

print("Downloading ACE-Step v1.5 Unified Repo (This includes DiT and LLM)...")
print("This may take a while (approx 10GB)...")

config_path = "acestep-v15-turbo"

# Use internal method if available to force download, otherwise init
try:
    if hasattr(handler, '_ensure_model_downloaded'):
        handler._ensure_model_downloaded(current_dir, config_path)
        print("\\n✅ Download complete.")
    else:
        print("Standard initialization...")
        handler.initialize_service(current_dir, config_path, device='cpu', offload_to_cpu=True)
except Exception as e:
    print(f"\\nDownload process finished (check for errors above if any): {e}")
"""

with open("download_models.py", "w") as f:
    f.write(download_code)

# 2. RUN DOWNLOAD
print("⏳ Starting Model Download... (This prevents timeouts)")
!python download_models.py

# 3. LAUNCH APP
print("="*60)
print("🚀 Models ready. Launching ACE-Step Interface...")
print("🔗 Click the public link ending in 'gradio.live' below once it appears.")
print("="*60)

!python app.py

⏳ Starting Model Download... (This prevents timeouts)
Initializing download sequence...
2026-02-27 09:37:27.207577: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1772185047.228694   25803 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1772185047.236022   25803 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1772185047.254425   25803 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1772185047.254467   25803 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.